# Vacation Search
UC Berkeley Extension Data Analytics Boot Camp Module 6 Challenge - Part 2

## Part 2 Instructions
#### Have Customers Narrow Their Travel Searches Based on Temperature and Precipitation

### Import the Data
#### Use the CSV file created in Part 1


In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [ ]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("WeatherPy_Database.csv")
city_data_df.head()

### Filter the DataFrame for minimum and maximum temperature preferences, and if the rain or snow accumulation is 0 inches or not using conditional statements. Do the following:
- Prompt the customer for the minimum temperature preference.
- Prompt the customer for the maximum temperature preference.
- Prompt the customer to answer if he or she would like it to be raining or not, using input("Do you want it to be raining? (yes/no) ").
- Prompt the customer to answer if he or she would like it to be snowing or not, using input("Do you want it to be snowing? (yes/no) ").

### Add the cities to a marker layer map with a pop-up marker for each city that includes:
- Hotel name
- City
- Country
- Current weather description with the maximum temperature

Save and upload the new DataFrame as WeatherPy_vacation.csv.

Save and upload the new marker layer map as WeatherPy_vacation_map.png.